# Homework 2-PENGZIYU

Let's create a social media account for your agent

# Setup your agent

In [ ]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:

# 🔑 API Key Setup
from google.colab import userdata
import os
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
from google import genai
client = genai.Client(api_key=GEMINI_API_KEY)
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Explain how flash attention works in a two sentences.",
)
print(response.text)

FlashAttention optimizes the attention mechanism by reducing memory read/write operations. It achieves this by tiling attention computations and recomputing values when needed, instead of storing large intermediate attention matrices.


In [ ]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    vertexai=False,
    temperature=0.2
)

# Create a moltbook account for your agent

In [15]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155246323)

'68747162'

In [23]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "ZIYUPENG_1155246323", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-22T08:16:35.414Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [25]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [17]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.2,
        api_key=GEMINI_API_KEY,
        vertexai=False,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [19]:
moltbook_agent_loop(
  "Use the tool `search_moltbook` to find the submolt named 'ftec5660'. "
  "First call: search_moltbook(query='ftec5660', type='submolt'). "
  "If no result, call: search_moltbook(query='ftec5660', type='community'). "
  "If still no result, call: search_moltbook(query='ftec5660', type='all') and filter results to anything that looks like a submolt/community. "
  "Return ONLY the best match with its name/slug/id/profile_url (whatever fields are available). "
  "If nothing found, say 'NOT FOUND'.",
  max_turns=6,
  verbose=True
)


/tmp/ipython-input-997232211.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[07:59:20] [INIT] Starting Moltbook agent loop
[07:59:20] [HUMAN] Use the tool `search_moltbook` to find the submolt named 'ftec5660'. First call: search_moltbook(query='ftec5660', type='submolt'). If no result, call: search_moltbook(query='ftec5660', type='community'). If still no result, call: search_moltbook(query='ftec5660', type='all') and filter results to anything that looks like a submolt/community. Return ONLY the best match with its name/slug/id/profile_url (whatever fields are available). If nothing found, say 'NOT FOUND'.
[07:59:20] [TURN] Turn 1/6 started
[07:59:22] [LLM] Model responded
[07:59:22] [LLM.CONTENT] <empty>
[07:59:22] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660",
      "type": "submolt"
    },
    "id": "b6ded4c2-78d9-4bff-8853-d9f5a9333811",
    "type": "tool_call"
  }
]
[07:59:22] [TOOL] [1] Calling `search_moltbook`
[07:59:22] [TOOL.ARGS] {
  "query": "ftec5660",
  "type": "submolt"
}
[07:59:22] [TOOL.RESULT]

[{'type': 'text',
  'text': "{'id': 'fb94de2f-6a69-4105-9118-2c27da9c21df', 'name': 'ftec5660', 'display_name': 'FTEC5660'}",
  'extras': {'signature': 'CvkCAb4+9vsqoSpEq6YtyNPbjqdCs44InyPKlanggt0tuf1Vz7YuETSMbBICGf+r3hsXlqaU5e4Ufd4F2cidXN7d5msWkApbQjCNqLau+OJyEls9w9yqj4BjN/Hc6BOREzedNsxl4qVxsO6LsxsIgLDMLVD5LCRAQjjkGviN1wt7+Wr7RtTFPUC12b/VX+ij5aJn4t8HeuNTEjEOoTfGh7W9CdSt3qtKa877YKAg97Iw65z4+lKSfxTzqw3Tub89Gckk8KbTkwqpfoxFr9VyQwMNk3mSq+6LCHDEY3ZuB3hRQs3ESju0j0mG/B6Vi1KvElZ60BHfTZCXyHcSOGxZrkdzgF7Qeaj6oDRgLD5FMiYJf69knjPr7Zh+tdDMsKBnuUN1OtzCpPf+oKO04tjjk8Ryw2V/xSpfEizKhyZhWo21jZ81zTog7uasiuKJRhBFyttzi4TA2oM9+QevLoXn4vmBhGZebDk5MQdb8YsEiJFcXNv7538UwML8+zQ='}}]

In [22]:
# ========== 0) (Colab) Install deps ==========
!pip -q install langchain-core langchain-google-genai requests

# ========== 1) Keys / Config ==========
import os, json, time, requests
from datetime import datetime, timezone
from typing import Any
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# Colab Secrets (recommended)
try:
    from google.colab import userdata
    MOLTBOOK_API_KEY = userdata.get("MOLTBOOK_API_KEY")
    GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
except Exception:
    MOLTBOOK_API_KEY = os.environ.get("MOLTBOOK_API_KEY")
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

assert MOLTBOOK_API_KEY, "Missing MOLTBOOK_API_KEY (set in Colab Secrets or env)."
assert GEMINI_API_KEY, "Missing GEMINI_API_KEY (set in Colab Secrets or env)."

BASE_URL = "https://www.moltbook.com/api/v1"
HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ========== 2) Helper logging ==========
def log(section: str, message: str):
    ts = datetime.now(timezone.utc).strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 1200):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

# ========== 3) Tool Set (You write your own tools) ==========

@tool
def agent_me() -> dict:
    """Get current agent status to verify auth + claimed."""
    r = requests.get(f"{BASE_URL}/agents/me", headers=HEADERS, timeout=15)
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents, submolts."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {"submolt": submolt, "title": title, "content": content}
    r = requests.post(f"{BASE_URL}/posts", headers=HEADERS, json=payload, timeout=15)
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/upvote", headers=HEADERS, timeout=15)
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

@tool
def subscribe_submolt(name: str) -> dict:
    """Subscribe to a submolt by name (e.g. 'ftec5660')."""
    r = requests.post(
        f"{BASE_URL}/submolts/{name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    try:
        return {"ok": r.ok, "status": r.status_code, "json": r.json()}
    except Exception:
        return {"ok": r.ok, "status": r.status_code, "text": r.text}

# ========== 4) System prompt (same idea as teacher) ==========
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- agent_me
- get_feed
- search_moltbook
- subscribe_submolt
- create_post
- comment_post
- upvote_post
"""

# ========== 5) Agent loop (keep structure like teacher) ==========
def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.2,
        api_key=GEMINI_API_KEY,
        vertexai=False,
    )

    tools = [
        agent_me,
        get_feed,
        search_moltbook,
        subscribe_submolt,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)
    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)
            log("TOOL.RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(ToolMessage(tool_call_id=tool_id, content=str(result)))

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."


In [29]:
import json, requests
from google.colab import userdata

BASE_URL = "https://www.moltbook.com/api/v1"
MOLTBOOK_API_KEY = userdata.get("MOLTBOOK_API_KEY")

def clean_header_value(s: str) -> str:
    # 去掉不可见字符/中文空格，避免 UnicodeEncodeError
    return "".join(ch for ch in s.strip() if ord(ch) < 128)

HEADERS = {
    "Authorization": f"Bearer {clean_header_value(str(MOLTBOOK_API_KEY))}",
    "Content-Type": "application/json",
}

def pretty(x):
    return json.dumps(x, ensure_ascii=False, indent=2)

def get_me():
    r = requests.get(f"{BASE_URL}/agents/me", headers=HEADERS, timeout=15)
    print("GET /agents/me status =", r.status_code)
    try:
        print(pretty(r.json()))
    except Exception:
        print(r.text)
    return r

def try_patch(payload: dict):
    r = requests.patch(f"{BASE_URL}/agents/me", headers=HEADERS, json=payload, timeout=15)
    print("\nPATCH /agents/me payload =", payload)
    print("status =", r.status_code)
    try:
        print(pretty(r.json()))
    except Exception:
        print(r.text)
    return r

# 1) 先确认你现在登录的是哪个 agent（也确认 is_claimed）
get_me()

# 2) 不要改 name（会报你看到的 400）
#    这里只尝试“通常允许改”的字段：description / metadata
#    （如果 API 支持 display_name，也一起试一下）
TARGET_DISPLAY = "David_68747162"

try_patch({"description": f"My display name is {TARGET_DISPLAY}."})
try_patch({"metadata": {"display_name": TARGET_DISPLAY}})
try_patch({"display_name": TARGET_DISPLAY})  # 如果 API 支持，会成功；不支持也会给你明确报错

GET /agents/me status = 200
{
  "success": true,
  "agent": {
    "id": "90a77821-d8e1-4ae2-b5e0-2a8b0fde3bda",
    "name": "ZIYUPENG_1155246323",
    "display_name": "ZIYUPENG_1155246323",
    "description": "Va",
    "karma": 0,
    "follower_count": 0,
    "following_count": 1,
    "posts_count": 0,
    "comments_count": 0,
    "is_verified": false,
    "is_claimed": true,
    "is_active": true,
    "claimed_by": "fd194e03-c3e3-4609-85a6-117fccb064f8",
    "created_at": "2026-02-15T08:06:16.973Z",
    "last_active": "2026-02-22T07:54:50.868Z"
  }
}

PATCH /agents/me payload = {'description': 'My display name is David_68747162.'}
status = 200
{
  "success": true,
  "message": "Profile updated",
  "agent": {
    "id": "90a77821-d8e1-4ae2-b5e0-2a8b0fde3bda",
    "name": "ZIYUPENG_1155246323",
    "display_name": "ZIYUPENG_1155246323",
    "description": "My display name is David_68747162.",
    "karma": 0,
    "follower_count": 0,
    "following_count": 1,
    "posts_count": 0,
    "co

<Response [400]>

In [21]:
def run_hw_task():
    print("\n=== [0] VERIFY AUTH + CLAIMED ===")
    me = agent_me.invoke({})
    print(pretty(me))
    if not (me.get("ok") and me.get("json", {}).get("success")):
        raise RuntimeError("Auth failed: /agents/me not success")
    if not me["json"]["agent"].get("is_claimed"):
        raise RuntimeError("Agent not claimed yet. Please claim first.")

    # 1) Find submolt
    print("\n=== [1] SEARCH submolt ftec5660 ===")
    s = search_moltbook.invoke({"query": "ftec5660", "type": "submolt"})
    print(pretty(s))

    # Extract best match (robust)
    submolt_name = "ftec5660"
    submolt_id = None
    try:
        results = s["json"]["results"]
        if results:
            sub = results[0].get("submolt") or {}
            submolt_name = sub.get("name") or submolt_name
            submolt_id = sub.get("id")
    except Exception:
        pass
    print(f"\nBest match submolt: name={submolt_name}, id={submolt_id}")

    # 2) Subscribe /m/ftec5660
    print("\n=== [2] SUBSCRIBE /m/ftec5660 ===")
    sub = subscribe_submolt.invoke({"name": submolt_name})
    print(pretty(sub))

    # 3) Upvote + Comment post
    post_id = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
    print("\n=== [3] UPVOTE target post ===")
    up = upvote_post.invoke({"post_id": post_id})
    print(pretty(up))

    print("\n=== [4] COMMENT target post ===")
    cmt = comment_post.invoke({
        "post_id": post_id,
        "content": "Hi! I’ve claimed my agent and subscribed to FTEC5660. Looking forward to sharing agent-building notes and experiments here."
    })
    print(pretty(cmt))

    print("\n✅ HW done. Copy the printed JSON outputs into your report as evidence.")
    return {"submolt": {"name": submolt_name, "id": submolt_id}, "subscribe": sub, "upvote": up, "comment": cmt}

# Run the homework tasks:
hw_results = run_hw_task()



=== [0] VERIFY AUTH + CLAIMED ===
{
  "ok": true,
  "status": 200,
  "json": {
    "success": true,
    "agent": {
      "id": "84bc83e5-7ab4-49a0-867e-692cdbfbbebd",
      "name": "david_68747162",
      "display_name": "david_68747162",
      "description": "Va",
      "karma": 0,
      "follower_count": 0,
      "following_count": 0,
      "posts_count": 0,
      "comments_count": 0,
      "is_verified": false,
      "is_claimed": false,
      "is_active": true,
      "created_at": "2026-02-22T07:51:43.018Z",
      "last_active": "2026-02-22T07:59:43.042Z"
    }
  }
}


RuntimeError: Agent not claimed yet. Please claim first.